In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import pandas as pd


In [2]:
# Load the data into a pandas DataFrame
data = pd.read_csv('../data/final_df.csv')

# Define Pydantic model
class Location(BaseModel):
    District: str
    Area: str
    Street: str
    Postal_Code: float
    Address: str
    Latitude: float
    Longitude: float

# Initialize FastAPI app
app = FastAPI()

# Endpoint to get all locations
@app.get("/locations/")
async def get_locations():
    return data.to_dict(orient="records")

# Endpoint to get a single location by index
@app.get("/locations/{location_id}")
async def get_location(location_id: int):
    try:
        return data.loc[location_id].to_dict()
    except KeyError:
        raise HTTPException(status_code=404, detail="Location not found")

# Endpoint to update a location
@app.put("/locations/{location_id}")
async def update_location(location_id: int, location: Location):
    try:
        data.loc[location_id] = location.dict()
        return {"message": "Location updated successfully"}
    except KeyError:
        raise HTTPException(status_code=404, detail="Location not found")

# Endpoint to delete a location
@app.delete("/locations/{location_id}")
async def delete_location(location_id: int):
    try:
        data.drop(location_id, inplace=True)
        return {"message": "Location deleted successfully"}
    except KeyError:
        raise HTTPException(status_code=404, detail="Location not found")

# Endpoint to create a new location
@app.post("/locations/")
async def create_location(location: Location):
    data = data.append(location.dict(), ignore_index=True)
    return {"message": "Location created successfully"}